## 1.	Hero:

Player of the board game, moving around the board step by step decided by the dices. Play can buy or conquer castles, discover equipments on the board. Heroes will die after lost all the gold and army. 

### Attribution:
name, position, attack, defense, army number, gold number, list of owned castles, equipment
### Function:
attack_castle (Hero, Castle), outputs the castle ownership, cost of army for the attacker, cost of army for the defender.
buy_castle (Hero, Castle), outputs the heroes’ gold amount, change of castle ownership.
hero_move (Hero, Dice), outputs new location of hero.

## 2.	Castles:
Main properties heroes can conquer or buy to own in the game. Castles provide certain number of gold and army to the owner heroes. 
### Attribution: 
position, owner, production, prize, location, stationed army

In [5]:
import random
import time
import sys
# '''Function modules'''
def hero_search(name,heros):
    """Input hero's name (string), heros list. output hero(class Hero) owns the name"""
    
    for hero in heros:
        if hero.name == name:
            return hero
            break
    else:
        return None  # if there is no one matches, return None
    
def roll_dice(dice_number):
    """Roll dices of dice_number"""
    i=1
    dice_sum = 0 
    input(">>Enter any to rolling dices.")
    print('Rolling dices......')
    time.sleep(1)
   
    while i <= dice_number:
        dice = random.randint(1, 6)
        print('Dice {} is {}.'.format(i, dice))
        dice_sum += dice
        i += 1    
    return dice_sum
        

In [8]:
# Class List

class getoutofloop(Exception):
    pass

class BoardGame:
    """Main part of the board game, store board, players list. Control game step, checking victory and delete failed player"""
    
    def __init__(self, num_player = 4):
        self.heros = [] # list of players
        self.board = [] # board information 
        self.size = num_player
        self.round = 1
        
    def round_check(self):  # hero with no gold, army, castle is lost, print notice, then remnove it from heroes list
        
        hero_next_round = []  # temp heroes survivor
        if self.size >= 2:
            for hero in self.heros:
                if hero.gold <= 0 and hero.army == 0 and len(hero.castle )< 1:
                    print("Hero "+ hero.name + " is defeated!\n" )
                    self.size -=1
                   
                else:
                    hero_next_round.append(hero)
                    self.heros = hero_next_round

        if self.size == 1:   # if there is only one hero left, he is the winner
            print("Hero "+  self.heros[0].name + " conquered the land!!!\n")  # end of the game
            time.sleep(1)
            print(" The End\n")
            time.sleep(1)
            while True:
                isexityes = input(">>Please input exit to quit\n")
                time.sleep(0.5)
                if isexityes.lower() == "exit":  # input exit to quit 
                    sys.exit()
                else:
                    continue 
                    
        else:  # check if want to end game in advance
            isexitadyes = input(">>Continue? Enter exit to quit, other key to continue.\n")
            time.sleep(0.5)
            if isexitadyes.lower() == "exit":  # input exit to quit in advance
                sys.exit()
              
            
    def round_step(self):  # board game running for a round
        
        # Notice of round start, all playr move and inactive with space
        print ("     [\n@XXXX[{::::::::::::::::::::::::::::::::::::>\n     [\n")
        time.sleep(0.5)
        print("Round " + str(self.round)+ "\n")
        time.sleep(1)
        self.round += 1
        
        for hero in self.heros:
            
            print(hero.name + ", it is your turn.\n")
            time.sleep(0.5)
            print("Currently, you have "+ str(hero.gold) + " gold, "+ str(hero.army) + " army, " + str(len(hero.castle)) +  " castles.\n")
            print("Your hero status is, Attack -- {} Defense -- {}.\n".format(hero.attack, hero.defense))  
           
            print("Equipment: ", end = "")# statement of hero status
            for equip in hero.equips:
                print(equip.name, end =", ")
            print("\n")
            time.sleep(0.5)
            hero.move_forward()

            space = hero.block_check(self.board)
            print(space)  # notice of where it is
            time.sleep(0.5)

            if type(space) == Castle:
                hero.act_castle(space, self.heros)
                
                
            elif type(space) == Arena:
                hero.act_arena()
                
            elif type(space) == Monolith:
                hero.act_monolith(space)
                
            elif type(space) == Market:
                hero.act_market(space)
                
                
            print("--------------------------------------------\n")
            time.sleep(1)
                
     
             
            
            # Hero collect production from the castles they owned 
            
        for hero in self.heros:
            if not hero.castle is None:
                for castle in hero.castle: 
                    hero.gold += castle.production
                    hero.army += castle.production
            else:
                continue   
            
        print("Heros received supply from their castles.\n")
        time.sleep(2)
        
        #print ("     [\n@XXXX[{::::::::::::::::::::::::::::::::::::>\n     [\n")
########################################################################################

class Building:
    """building on the board, including castles, shop, etc"""

    def __init__(self, name, location):
        self.name = name
        self.location = location


class Castle(Building):
    
    # Building can be purchased or captured by Hero, class hero def act_castle

    def __init__(self, name, location, price):
        super().__init__(name, location)

        self.level = 1  # maximum level is 5
        self.price = price
        self.up_fee = int(price * 0.6)  # upgrade cost for each time
        self.up_ratio = 2  # production improvement for each upgrade
        self.owner = None  # None at begining
        self.army = price
        self.production = int(price / 20)  # production of gold and army per day
        
    def __str__(self):
        return "Here is a Level " + str(self.level) + " Castle "+ self.name + ".\n(Price: "+ str(self.price) + " Army : "+ str(self.army)+")"  
    # building welcome word
        
class Arena(Building):
    
    # Building where heroes can be trained, class hero def act_arena
    
    def __init__(self, name, location):
        super().__init__(name, location)
        
    def __str__(self):
        return "Here is Arena " + self.name + ".\n"  # building welcome word
    
class Monolith(Building):
    
    # Building where hero teleported to self.next_position, class hero def act_monolith
    
    def __init__(self, name, location, next_location):
        super().__init__(name, location)
        self.next_location = next_location 
        
    def __str__(self):
        return "Here is a Monolith " + self.name + ".\n"  # building welcome word
    

class Market(Building):
    
    # Building where hero can purchase equipment, class hero def act_market
    
    def __init__(self, name, location):
        super().__init__(name, location)
        self.equips_stk = []  # equipments in stock at the market 
        
    def __str__(self):
        return "Here is " + self.name + " Market.\n"  # building welcome word


########################################################################################

class Equipment:
    
    # Equipment stock at the sotre, be purchased and equiped by Hero, class hero def act_market
    
    def __init__(self, name, price, att_add, def_add):
        self.name = name
        self.price = price
        self.att_add = att_add  # damage increase percentage in battle 
        self.def_add = def_add  # damage received decrease percentage in battle 
    
    def __str__(self):
        return '{:35s}Attack:{:4d}, Defense:{:4d} ----- Gold {:4d}'.format(self.name, self.att_add, self.def_add, self.price)

########################################################################################

class Hero:
    
    """Player is a hero."""

    def __init__(self, name, gold = 1500,army = 1500):

        self.name = name
        self.location = 0
        self.gold = gold  #1500
        self.army = army  #1500
        self.castle = []
        self.equips = []
        self.attack = 100  
        self.defense = 100
        

    def move_forward(self):
        """Movement decided by two dices""" 
        forward = roll_dice(2)   # roll two dices once
    
        self.location += forward  # update position after movement

        if self.location >= 40:
            self.gold += 200  # pass go collect 200 gold
            print('Collect 200 gold as you pass go.\n')
            time.sleep(0.5)
        self.location = self.location % 40  # board is of 40 blocks

        print('Hero ' + self.name + ' is marching, arrives Space ' + str(
            self.location) + ' now.\n')  # location update info
        time.sleep(0.5)
        

    def block_check(self, board):

        """input is hero, and board is list of all the blocks,
           output the type of block"""

        for space in board:
            if self.location == space.location:
                building = space
                break
            else:
                continue

        return building  # return the building where hero is located: castle, shop, etc
    
    # (optional )def surrender(self): """Hero choose to surrender, empty all the castle ownership, and quit the game"""
        
    
    def act_arena(self):
        
        """Actions heroes can take at Arena, do some training to enhance attak and defence,decided by dices"""
        
        while True:
            istrainyes = input(
                ">>Any fight art training here? Your soldiers will get wounded but you will be stronger? (Y - Yes, N - No)\n")
            
            if istrainyes.lower() == 'y':
                train_cost = roll_dice(2)
                self.attack += train_cost
                self.defense += train_cost
                self.army -= train_cost*10
                
                print("Your attack, defense increased by "+ str(train_cost)+'.\n')
                time.sleep(1)
                
                if self.army < 0:  # mim army size is 0
                    self.army = 0
                break
            
            elif istrainyes.lower() == 'n':
                print("Maybe next time.\n")
                time.sleep(0.5)
                break
                
            else:
                print("Pardon me.\n")
                time.sleep(0.5)
    
    def act_monolith(self, monolith):
        
        """Hero teleport to monolith.next_location  
        """
        
        print("After touching the Monolith, Hero and the troops are teleported by some unknown power...\n")
        time.sleep(0.5)
        self.location = monolith.next_location
        print("You are at space " + str(self.location) + " now.\n")
        time.sleep(0.5)
        
    def act_market(self, market):
        
        """Hero can purchase equipment from the market.equips_stk
        """ 
        try:
            while True:
                ispurchaseyes = input(">>Would you like to buy any equipment(One item limit every visit)? (Y - Yes, N - No)\n")
                time.sleep(0.5)
                
                if ispurchaseyes.lower() == "y":  # want to buy
                    print("Welcome, here is the sale list today!\n")
                    time.sleep(0.5)
                                  
                    for i in range(len(market.equips_stk)):
                        print(i,end = "-")
                        print(market.equips_stk[i])
                    print("Q-Quit")
                    time.sleep(0.5)  ## print sale list
                    
                    while True:
                        equip_id = input(">>Which equipment you want to purchase? (Input NO.of the equipment to choose, Q - Quit)\n")
                        time.sleep(0.5)
                        if equip_id.lower() == 'q':
                            raise getoutofloop()  ## jump out of two whiles
                            
                        elif equip_id.isdigit():  ## input is number
                            equip_id = int(equip_id)
                            if not equip_id in range(len(market.equips_stk)): # not available item
                                print("Pardon me. Please make the available choice.\n")
                                time.sleep(0.5)
                                continue  # continue for rechoice id
                                  
                            elif self.gold < market.equips_stk[equip_id].price:
                                print("You don't have enough gold.Please make the available choice.\n")
                                time.sleep(0.5)
                                continue  # continue for rechoice id
                        
                            elif self.gold >= market.equips_stk[equip_id].price:  # available item, enough gold
                                self.gold -= market.equips_stk[equip_id].price   
                                self.attack += market.equips_stk[equip_id].att_add 
                                self.defense += market.equips_stk[equip_id].def_add
                                self.equips.append(market.equips_stk.pop(equip_id))  # equipment add to hero.equips list, pop from the equips_stk list
                                print("You received a new equipment!\n")
                                time.sleep(0.5)
                                raise getoutofloop()  # jump out of two whiles
                                
                        else:
                            print("Pardon me. Please make the available choice.\n")
                            continue
                            
                elif ispurchaseyes.lower() == "n":  # don't want to buy
                    time.sleep(0.5)
                    break  # equip the outsides while
                
                else:  # unrecognize input
                    print("Pardon me.\n")
                    time.sleep(0.5)
                    continue  # continue choose if want to buy
                                               
        except getoutofloop:
            pass
        
        print("See you next time\n")
                                            
             
        
    

    def act_castle(self, castle, heros):
        """Actions heroes can take at a castleinput castle are castle hero is located, heros is list of heroes.(BoardGame.heros)"""

        if castle.owner is None:  # castle is of no owner it is available for sale

            while True:

                isbuyyes = input(
                    ">>Would you like to purchase the Castle for " + str(castle.price) + " gold? (Y - Yes, N - No)\n")

                if isbuyyes.lower() == "y":  # want to buy

                    if self.gold >= castle.price:  # enough gold
                        self.gold -= castle.price
                        castle.owner = self.name
                        self.castle.append(castle)
                        print("Your territory just expanded!\n")
                        time.sleep(0.5)
                        break

                    else:
                        print("You don't have enough gold! Purchase failed.\n")
                        time.sleep(0.5)
                        break

                elif isbuyyes.lower() == "n":  # don't want to buy
                    print("You and your soldiers stationed outsides the castle for one night.\n")
                    time.sleep(0.5)
                    break

                else:
                    print("Pardon me.\n")
                    time.sleep(0.5)
                    continue

        elif castle.owner == self.name:  # Heroes can upgrade the castle that belongs to himself.

            if castle.level == 5:
                print(" Your highness, the castle has been upgraded to the top level!\n")
                time.sleep(1)

            else:
                while True:
                    
                    islevelyes = input(
                        ">>Would you like to upgrade the Castle for " + str(castle.up_fee) + " gold? (Y - Yes, N - No)\n")
                    time.sleep(0.5)

                    if islevelyes.lower() == "y":  # want to upgrade

                        if self.gold >= castle.up_fee:  # enough gold
                            self.gold -= castle.up_fee  # charge
                            castle.level += 1  # upgrade level
                            castle.army *= castle.up_ratio
                            castle.production *= castle.up_ratio
                            
                            print("Your castle has reached level " + str(castle.level) + " !\n")
                            time.sleep(0.5)
                            break

                        else:  # not enough gold
                            print("You don't have enough gold! Upgrade failed.\n")
                            time.sleep(0.5)
                            break

                    elif islevelyes.lower() == "n":  # don't want to upgrade
                        print("Wish you bring some reinforce next time.\n")
                        time.sleep(0.5)
                        break

                    else:
                        print("Pardon me.\n")
                        time.sleep(0.5)
                        continue
                        
        else:  # Heros can try to capture the castle that belongs to others.
            while True:
                isattackyes = input(
                        ">>This is a castle belongs to enemy. Caputre it? (Y - Yes, N - No)\n")
                
                if isattackyes.lower() == "y":  # want to attack 
                    print("Battle Starts!!!\n")
                    time.sleep(1)
                    
                    hero_power = self.army * (1 + self.attack)/(1 + castle.level * 20)  # hero power  
                    castle_power = castle.army * 10 * castle.level / (self.defense / 100)  # castle power during battle
                    
                    if hero_power == castle_power:  # it is a tie, hero lost all the army
                        self.army = 0
                        print("The battle is over.We didn't make it... fought until the final soldier.\n")
                        time.sleep(0.5)
                        break
                        
                    elif hero_power < castle_power:  # hero lost
                        self.army = 0
                        winner = hero_search(castle.owner,heros) # winner is owner of the castle, 
                        winner.gold += castle.price  # winner get the gold
                        winner.gold = int(winner.gold)
    
                        self.gold -= castle.price # pay gold castle price to the owner of castle
                        self.gold = int(self.gold)
                        print("The enemy is too strong.Our troops are defeated. We paid some gold as indemnity.\n")
                        time.sleep(0.5)
                        break
                        
                    else:  # hero win
                        army_lost = int(self.army*castle_power/hero_power)
                        self.army -= army_lost
                        castle.owner = self.name
                        self.castle.append(castle)  # hero get the castle
                        
                        loser = hero_search(castle.owner,heros)  # castle owner is the loser
                        loser.castle.remove(castle)  # loser lost the owner
                        print("Victory! We caputred another castle, my lord!\n")
                        time.sleep(0.5)
                        break
                    
                elif isattackyes.lower() == "n":  # no attack, just pass
                    print("You troop sneaked out of the enemy's terrtory safely.\n")
                    time.sleep(0.5)
                    break
                else:
                    print("Pardon me.\n")
                    time.sleep(0.5)
                    continue
            

In [23]:
# Initial Equipment & Market

weapon_market= Market("Weapon",10)

weapon_market.equips_stk.append(Equipment("Blackshard of the Dead Knight",300, 30, 0))  # weapon
weapon_market.equips_stk.append(Equipment("Red Dragon Flame Tongue",400, 20, 20)) 
weapon_market.equips_stk.append(Equipment("Sword of Hellfire",600, 60, 0))
weapon_market.equips_stk.append(Equipment("Titan`s Gladius",1000, 120, -30))
weapon_market.equips_stk.append(Equipment("Sword of Judgement",2000, 100, 100))  

weapon_market.equips_stk.append(Equipment("Shield of the Yawning Dead", 300, 0, 30))   # shield
weapon_market.equips_stk.append(Equipment("Targ of the Rampaging Ogre", 500, 0, 50))
weapon_market.equips_stk.append(Equipment("Dragon Scale Shield", 600, 30, 30))
weapon_market.equips_stk.append(Equipment("Sentinel`s Shield", 1000, -30, 120))
weapon_market.equips_stk.append(Equipment("Lion`s Shield of Courage", 1600, 80, 80))


armor_market = Market("Armor",30)

armor_market.equips_stk.append(Equipment("Scales of the Greater Basilisk", 300, 30, 0))  # armor
armor_market.equips_stk.append(Equipment("Dragon Wing Tabard", 400, 20, 20))
armor_market.equips_stk.append(Equipment("Tunic of the Cyclops King", 500, 0, 40))
armor_market.equips_stk.append(Equipment("Dragon Scale Armor", 800, 40, 40))
armor_market.equips_stk.append(Equipment("Titan`s Cuirass", 1000, -20, 100))

armor_market.equips_stk.append(Equipment("Crown of the Supreme Magi", 500, 40, 0))  # helmet
armor_market.equips_stk.append(Equipment("Hellstorm Helmet", 600, 50, 0))
armor_market.equips_stk.append(Equipment("Crown of Dragontooth", 800, 40, 40))
armor_market.equips_stk.append(Equipment("Thunder Helmet", 1000, 100, -20))
armor_market.equips_stk.append(Equipment("Helm of Heavenly Enlightenment", 2400, 120, 120))



>>Would you like to buy any equipment(One item limit every visit)? (Y - Yes, N - No)
y
Welcome, here is the sale list today!

0-Scales of the Greater Basilisk     Attack:  30, Defense:   0 ----- Gold  300
1-Dragon Wing Tabard                 Attack:  20, Defense:  20 ----- Gold  400
2-Tunic of the Cyclops King          Attack:   0, Defense:  40 ----- Gold  500
3-Dragon Scale Armor                 Attack:  40, Defense:  40 ----- Gold  800
4-Titan`s Cuirass                    Attack: -20, Defense: 100 ----- Gold 1000
5-Crown of the Supreme Magi          Attack:  40, Defense:   0 ----- Gold  500
6-Hellstorm Helmet                   Attack:  50, Defense:   0 ----- Gold  600
7-Crown of Dragontooth               Attack:  40, Defense:  40 ----- Gold  800
8-Thunder Helmet                     Attack: 100, Defense: -20 ----- Gold 1000
9-Helm of Heavenly Enlightenment     Attack: 120, Defense: 120 ----- Gold 2400
Q-Quit
>>Which equipment you want to purchase? (Input NO.of the equipment to choose,

In [ ]:
## Test hero.act_market
p1 = Hero("张飞",10000000, 1000)
p2 = Hero("关羽",0,0)

for i in range(2):
    p2.act_market(armor_market)

for equip in armor_market.equips_stk:
    print(equip)
    
print()

for equip in p1.equips:
    print(equip)

In [ ]:
# market list check
for i in range(len(weapon_market.equips_stk)):
    print(i,end = "-")
    print(weapon_market.equips_stk[i])
            
print()

for i in range(len(armor_market.equips_stk)):
    print(i,end = "-")
    print(armor_market.equips_stk[i])

In [7]:
####################################################################
# Initial and Play a new game
print("Welcome to Heroes of Might & Magic Monopoly.\n")
time.sleep(0.5)

while True:
    player_num = input(">>How many players(Heroes) will play the game? (2-4 is recommeded)\n")  # decide players nubmer
    time.sleep(0.5)
    
    if player_num.isdigit():
        player_num = int(player_num)
        if player_num < 2:
            print("At least 2 players are required.\n")
            time.sleep(0.5)
            continue
        elif player_num > 6:
            print("Too many players will affect the playing experience.")
            time.sleep(0.5)
            continue
        else:
            break
    else:
        print("This is not number.\n")
        continue

newgame = BoardGame(player_num)  # initial newgame class BoardGame
# initial work for players
namelist = []  # waiting list for heroes names
for i in range(player_num):
    while True:
        hero_name = input(">>Play {}, enter your hero's name.".format(i+1))
        time.sleep(0.5)
        if hero_name == "":  # enter unitl an unempty name
            print("Blank name is not accepted.\n")
            time.sleep(0.5)
            continue
        elif hero_name in namelist:  # hero can't have the same name
            print("Other alway has the same name.\n")
            time.sleep(0.5)
        else:
            newgame.heros.append(Hero(hero_name))
            namelist.append(hero_name)
            break
            

# initial board map, 40 blocks
for i in range(0,20):   
    newgame.board.append(Castle("Hefei",i,1000))
for i in range(20,40):
    newgame.board.append(Monolith("Unknown",i,(i+10)%40))

# play the game
while True:
    newgame.round_step()
    newgame.round_check()
    
    

Welcome to Heroes of Might & Magic Monopoly.

>>How many players(Heroes) will play the game? (2-4 is recommeded)
2
>>Play 1, enter your hero's name.
Blank name is not accepted.

>>Play 1, enter your hero's name.d
>>Play 2, enter your hero's name.a
     [
@XXXX[{::::::::::::::::::::::::::::::::::::>
     [

Round 1

d, it is your turn.

Currently, you have 1500 gold, 1500 army, 0 castles.

Your hero status is, Attack -- 100 Defense -- 100.

Equipment: 

>>Enter any to rolling dices.
Rolling dices......
Dice 1 is 5.
Dice 2 is 4.
Hero d is marching, arrives Space 9 now.

Here is a Level 1 Castle Hefei.
(Price: 1000 Army : 1000)
>>Would you like to purchase the Castle for 1000 gold? (Y - Yes, N - No)
y
Your territory just expanded!

--------------------------------------------

a, it is your turn.

Currently, you have 1500 gold, 1500 army, 0 castles.

Your hero status is, Attack -- 100 Defense -- 100.

Equipment: 

>>Enter any to rolling dices.
Rolling dices......
Dice 1 is 3.
Dice 2 is 1.

SystemExit: 

C:\Users\Haoyu Zhang\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
####################################################################
# Check victory, end of game
print("Welcome to Heroes of Might & Magic Monopoly.\n")
time.sleep(0.5)



newgame = BoardGame(2)  # initial newgame class BoardGame
# initial work for players

newgame.heros.append(Hero("赵云"))
newgame.heros.append(Hero("关羽", 0, 0))     
            

# initial board map, 40 blocks
for i in range(0,20):   
    newgame.board.append(Castle("Hefei",i,1000))
for i in range(20,40):
    newgame.board.append(Monolith("Unknown",i,(i+10)%40))

# play the game
while True:
    newgame.round_step()
    newgame.round_check()
    

Welcome to Heroes of Might & Magic Monopoly.

     [
@XXXX[{::::::::::::::::::::::::::::::::::::>
     [

Round 1

赵云, it is your turn.

Currently, you have 1500 gold, 1500 army, 0 castles.

Your hero status is, Attack -- 100 Defense -- 100.

Equipment: 

>>Enter any to rolling dices.y
Rolling dices......
Dice 1 is 4.
Dice 2 is 4.
Hero 赵云 is marching, arrives Space 8 now.

Here is a Level 1 Castle Hefei.
(Price: 1000 Army : 1000)
>>Would you like to purchase the Castle for 1000 gold? (Y - Yes, N - No)
y
Your territory just expanded!

--------------------------------------------

关羽, it is your turn.

Currently, you have 0 gold, 0 army, 0 castles.

Your hero status is, Attack -- 100 Defense -- 100.

Equipment: 

>>Enter any to rolling dices.
Rolling dices......
Dice 1 is 3.
Dice 2 is 3.
Hero 关羽 is marching, arrives Space 6 now.

Here is a Level 1 Castle Hefei.
(Price: 1000 Army : 1000)
>>Would you like to purchase the Castle for 1000 gold? (Y - Yes, N - No)
y
You don't have enough gold!

SystemExit: 

C:\Users\Haoyu Zhang\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [15]:
# import map.csv

import csv

map_csv_read = open('map.csv', 'rt')
csvin = csv.reader(map_csv_read)


for row in csvin:
    if row[0] == "Castle":
        name, loc, price = row[1], int(row[2]), int(row[3])
        # newgame.board.append(Castle("Cloudfire", 1, 60))
        print(type(name))
        print(type(loc))
        print(type(price))
        print()
    
    elif row[0]== "Market":
        print(row[1])
    
    elif row[0]=="Arena":
        print(row[1])
    
    else:
        print(row[1])
        
map_csv_read.close()

Weapon
<class 'str'>
<class 'int'>
<class 'int'>

Empire
<class 'str'>
<class 'int'>
<class 'int'>

Unknown
<class 'str'>
<class 'int'>
<class 'int'>

War
<class 'str'>
<class 'int'>
<class 'int'>

<class 'str'>
<class 'int'>
<class 'int'>

Helmet
<class 'str'>
<class 'int'>
<class 'int'>

<class 'str'>
<class 'int'>
<class 'int'>

Unknown
<class 'str'>
<class 'int'>
<class 'int'>

<class 'str'>
<class 'int'>
<class 'int'>

Magic
<class 'str'>
<class 'int'>
<class 'int'>

<class 'str'>
<class 'int'>
<class 'int'>

Shield
<class 'str'>
<class 'int'>
<class 'int'>

<class 'str'>
<class 'int'>
<class 'int'>

Marletto
<class 'str'>
<class 'int'>
<class 'int'>

<class 'str'>
<class 'int'>
<class 'int'>

Unknown
<class 'str'>
<class 'int'>
<class 'int'>

<class 'str'>
<class 'int'>
<class 'int'>

Armor
<class 'str'>
<class 'int'>
<class 'int'>

<class 'str'>
<class 'int'>
<class 'int'>

Revelation
<class 'str'>
<class 'int'>
<class 'int'>

Unknown
Enlightenment
<class 'str'>
<class 'int'>
<c

In [ ]:
## 3.	Board:
Main map records heroes in play, location of castles, equipment, shop(optional)
### Attribution: 
list of heroes, list of castles, list of equipment
### Fuction: 
round_step, moving to the next round after all players’ activities, list
heroes’ details: location, castle capture, army, gold; print map of board. 

## 4.	Dice:
To decide movement of heroes and random event.
### Attribution: 
numbers 
### Function: 
roll_dice (Hero, Dice), outputs steps hero will move in the current round.